# Code for Examining Individual Fit Parameters from the Cosine Moment Extractions

In [1]:
%reset -f
import ROOT
import traceback
from MyCommonAnalysisFunction_richcap import *

# ROOT.TH1.AddDirectory(0)
# ROOT.gStyle.SetTitleOffset(1.3,'y')

# ROOT.gStyle.SetGridColor(17)
# ROOT.gStyle.SetPadGridX(1)
# ROOT.gStyle.SetPadGridY(1)

print(f"{color.BOLD}\nStarting RG-A SIDIS Analysis\n{color.END}")

Welcome to JupyROOT 6.22/06

Starting RG-A SIDIS Analysis



.

.

.

## Specify the Parameter text file to use:

In [2]:
# Define the relative path to the file
file_path = 'SIDIS_python_Images_From_5D_Unfold_Test_V6_5_21_2024/Parameters_5D_Unfold_Test_V6_5_21_2024_Q2_y_Bins_Combined.txt'  # Adjust the path based on your specific directory structure
file_path = 'SIDIS_python_Images_From_Sim_Test_Pass_2_5D_Unfold_Test_V6_5_24_2024/Parameters_Sim_Test_Pass_2_5D_Unfold_Test_V6_5_24_2024_Q2_y_Bins_Combined.txt'

file_path = 'Parameters_Pass_2_New_Sector_Cut_Test_V3_6_4_2024_Q2_y_Bins_3_Unsmeared.txt'

# Open the file and read its contents into a string
with open(file_path, 'r') as file:
    file_content = file.read()

print(f"{color.BOLD}File: {color.BLUE}{file_path}{color.END_B} has been found...{color.END}")

# # Now `file_content` holds the content of the file as a string
# print(file_content)  # Display the content to verify it's correct


File: Parameters_Pass_2_New_Sector_Cut_Test_V3_6_4_2024_Q2_y_Bins_3_Unsmeared.txt has been found...


In [17]:
# print(file_content)  # Display the content to verify it's correct

.

.

.

## Declare Pass Version

In [3]:
Pass_Version = "Pass 1"
Pass_Version = "Pass 2" if(any(pass_V in str(file_path) for pass_V in ["Pass_2", "P2"])) else Pass_Version
# Pass_Version = "Pass 2"

print(f"\n{color.BOLD}Pass Version in use = {color.UNDERLINE}{Pass_Version}{color.END}\n")


Pass Version in use = Pass 2



.

.

.

### Remove User text
(Attempts to limit the lines of text included for humans reading the content of the loaded file so that the computer has an easier time searching through the file)

In [4]:
file_content = file_content.replace(f"This information is from {color.BOLD}{Pass_Version}{color.END}", "")
file_content = file_content.replace("""
Note to Reader: Print the text in this file as a string in Python for the best formatting...


""", "")

print(f"\n{color.BOLD}'file_content'{color.END} has been updated\n")


'file_content' has been updated



.

.

.

In [6]:
# def extract_histogram_info(block, histogram_type):
#     pattern = rf"\(\*\) {re.escape(histogram_type)} Histograms:(.*?)(?=\(\*\)|$)"
#     match = re.search(pattern, block, re.DOTALL)
#     if(match):
#         return match.group(1).strip()
#     else:
#         return "No data found for the specified histogram type."

# block = """For Q2-y Bin 3 - z-PT Bin 1: 
#      (*) 1D Histograms:
#      - Bin-by-Bin Correction Fits:
#              Par A    = 13486.068651293246 ± 54.70518618985534
#              Par B    = -0.08165271387166789 ± 0.005685585802843861
#              Par C    = 0.031354864785378384 ± 0.005653182085145293
#              chi2/NDF = 8.1961606104412
#      - Bayesian Unfolding Fits:
#              Par A    = 13941.171993298445 ± 85.64670061373363
#              Par B    = -0.08593225635958629 ± 0.008326754543945727
#              Par C    = 0.027749049170228628 ± 0.008161871933343137
#              chi2/NDF = 8.039574428843164
#      - Generated Plot Fits:
#              Par A    = 3281.6052095412715 ± 11.693213232322705
#              Par B    = 0.0015283478118269472 ± 0.0008860486093057907
#              Par C    = 0.00799254726191877 ± 0.005900899459140923
#              chi2/NDF = 1.0702373297695
#      (*) 1D (pipsec 1) Histograms:
#      - Bin-by-Bin Correction Fits:
#              Par A    = 2287.1860384785923 ± 53.87622224466281
#              Par B    = -0.10488598521460385 ± 0.030987473593416803
#              Par C    = 0.04439845524906173 ± 0.03329647340756958
#              chi2/NDF = 0.8207842620741114
#      - Bayesian Unfolding Fits:
#              Par A    = 2287.1860384785923 ± 53.87622224466281
#              Par B    = -0.10488598521460385 ± 0.030987473593416803
#              Par C    = 0.04439845524906173 ± 0.03329647340756958
#              chi2/NDF = 0.8207842620741114
#      - Generated Plot Fits:
#              Par A    = 546.0954045256018 ± 4.769870072131425
#              Par B    = 0.019408575607271 ± 0.014351610748536957
#              Par C    = 0.020489649964164684 ± 0.015036170802304032
#              chi2/NDF = 1.4623952555983086"""
# print(extract_histogram_info(block, histogram_type="1D (pipsec 1)"))
# print(extract_histogram_info(block, histogram_type="1D"))

## Defining Search Functions

In [7]:
def find_bin_block(data, bin_info):
    blocks = re.split(r"==+\n", data)
    for block in blocks:
        if(bin_info in block):
            if((("SMEARED" in bin_info) and ("SMEARED" in block)) or (("SMEARED" not in bin_info) and ("SMEARED" not in block))):
                return f"\n======================================================================\n{block.strip()}\n======================================================================\n"
    return "No data found for the specified Q2-y/z-PT Bin."


def extract_histogram_info(block, histogram_type):
    # pattern = rf"{histogram_type} Histograms:(.*?)(?=\(\*\)|\dD Histograms)"
    pattern = rf"\(\*\) {re.escape(histogram_type)} Histograms:(.*?)(?=\(\*\)|$)"
    match = re.search(pattern, block, re.DOTALL)
    if(match):
        return match.group(1).strip()
    else:
        return "No data found for the specified histogram type."
    

def extract_fit_info(block, fit_type):
    pattern = rf"- {re.escape(fit_type)} Fits:.*?(\n\s+Par A.*?chi2/NDF = \S+)"
    match = re.search(pattern, block, re.DOTALL)
    if(match):
        return f"\n{match.group(1).strip()}"
    else:
        return "No fit details found for the specified type."

    
# def find_parameter(block, parameter, return_err=False):
#     for line in block.split("\n"):
#         if(parameter in line):
#             par_val = (line.split("="))[1]
#             par_val = re.sub(r"{}.*".format(re.escape("±")), "", par_val)
#             return par_val
#     return "ERROR"

def find_parameter(block, parameter, return_err=False):
    for line in block.split("\n"):
        if(parameter in line):
            try:
                # Split the line at "=", then trim whitespace around the parts
                _, par_info = line.split("=")
                par_val, par_err = map(str.strip, par_info.split("±"))
                if(return_err):
                    return par_val, par_err
                else:
                    return par_val
            except ValueError:
                # If splitting fails, likely due to missing '=' or '±'
                continue
    if(return_err):
        return "ERROR", "ERROR"
    else:
        return "ERROR"
            
def Full_Search_Parameter(Parameter_File_str, Bin_Info=False, Histogram_Type=False, Fit_Type=False, Parameter=False, return_err=False):
    Output = "Error"
    if(not Bin_Info):
        print("No kinematic bin given")
        return Parameter_File_str
    else:
        Output = find_bin_block(Parameter_File_str, Bin_Info)
        if(not Histogram_Type):
            print("No histogram given")
            return Output
        else:
            Output = extract_histogram_info(Output, Histogram_Type)
            if(not Fit_Type):
                print("No fit given")
                return Output
            else:
                Output = extract_fit_info(Output, Fit_Type)
                if(not Parameter):
                    print("No parameter given")
                    return Output
                else:
                    Output = find_parameter(Output, Parameter, return_err)
                    return Output
    return Output
    
print("Done making search functions")

Done making search functions


.

.

.

# Create string of C++ Code for the Moment measurements

In [6]:
String_Print_Cpp = """
float Moment_Measurement[18][37][2][2];

    // Moment_Measurement[Q2_y_Bin][z_pT_Bin][Moment_Num][Smear_Num]
    // Moment_Num = 0 -> Cos(phi)   (Fit Par B)
    // Moment_Num = 1 -> Cos(2*phi) (Fit Par C)
    // Smear_Num  = 0 -> NOT Smeared
    // Smear_Num  = 1 -> Smeared"""
for Q2_y_Bin in range(1, 18):
    if(Q2_y_Bin > 1):
        String_Print_Cpp = f"{String_Print_Cpp}\n"
    for z_pT_Bin in range(1, Get_Num_of_z_pT_Bins_w_Migrations(Q2_y_Bin_Num_In=Q2_y_Bin)[1] + 1):
        Num_Spaces  = " "
        for ii in [Q2_y_Bin, z_pT_Bin]:
            if(ii < 10):
                Num_Spaces += " "
        for Smearing in ["SMEARED ", ""]:
            String_Print_Cpp = f"{String_Print_Cpp}\n"
            if(Smearing in ["SMEARED "]):
                Smear_Num = "1"
                String_Print_Cpp = f"{String_Print_Cpp}\n// (Q2-y) + (z-pT) Bin: ({Q2_y_Bin}) + ({z_pT_Bin})"
            else:
                Smear_Num = "0"
            Bin___Type = f"{Smearing}Q2-y Bin {Q2_y_Bin} - z-PT Bin {z_pT_Bin}"
            Histo_Type = "1D"
            Fit___Type = "Bin-by-Bin Correction"
            if(skip_condition_z_pT_bins(Q2_Y_BIN=Q2_y_Bin, Z_PT_BIN=z_pT_Bin, BINNING_METHOD="Y_bin")):
                for Par___Type in ["B", "C"]:
                    # Moment_Value = "-0.1500" if(Par___Type in ["B"]) else "0.005"
                    Moment_Value = "N/A" if(Par___Type in ["B"]) else "N/A"
                    Moment_Num   =  0        if(Par___Type in ["B"]) else 1
                    String_Print_Cpp = f"""{String_Print_Cpp}
Moment_Measurement[{Q2_y_Bin}][{z_pT_Bin}][{Moment_Num}][{Smear_Num}]{Num_Spaces}=  {Moment_Value};"""
            else:
                for Par___Type in ["B", "C"]:
                    Moment_Value = Full_Search_Parameter(Parameter_File_str=file_content, Bin_Info=Bin___Type, Histogram_Type=Histo_Type, Fit_Type=Fit___Type, Parameter=Par___Type)
                    Moment_Value = str(round(float(Moment_Value), 5))
                    Moment_Num = 0 if(Par___Type in ["B"]) else 1
                    String_Print_Cpp = f"""{String_Print_Cpp}
Moment_Measurement[{Q2_y_Bin}][{z_pT_Bin}][{Moment_Num}][{Smear_Num}]{Num_Spaces}=  {Moment_Value};"""
            
String_Print_Cpp = f"{String_Print_Cpp}\n"
String_Print_Cpp = String_Print_Cpp.replace(";\n", "; \t")
String_Print_Cpp = String_Print_Cpp.replace("  -",  " -")
# String_Print_Cpp = String_Print_Cpp.replace("  ",   " ")


print(f"\n{String_Print_Cpp}\n")

# String_Print_python = String_Print_Cpp.replace("""float Moment_Measurement[18][37][2]; 	
#     // Moment_Measurement[Q2_y_Bin][z_pT_Bin][Moment_Num]
#     // Moment_Num = 0 -> Cos(phi)   (Fit Par B)
#     // Moment_Num = 1 -> Cos(2*phi) (Fit Par C)
#
# """, "")
# String_Print_python = String_Print_python.replace("][", ", ")
# String_Print_python = String_Print_python.replace("[", "['")
# String_Print_python = String_Print_python.replace("]", "']")
# String_Print_python = String_Print_python.replace("; \t", "\n")
# print(String_Print_python)

ValueError: could not convert string to float: 'ERROR'

.

.

.

# Comparison between Fits

## Compare Smearing

In [162]:
Histo_Type = "1D"
Fit___Type = "Bin-by-Bin Correction"
Parameters_to_Compare = ["B", "C"]

Comparison_Output = {}
Comparison_Output["Title"] = ["Q2-y-z-PT Bin", "Fit Parameter",  "Unsmeared\t\t", "Smeared\t\t", "Absolute Dif\t", "Percent Dif\t\t"]
# Comparison_Output["Title2"] = ["Bin 18-35\t",  "Parameter B\t ", "0.15\t",    "0.16\t",   "0.01\t", "6.7%\t"]
# Comparison_Output["Title3"] = ["Bin 1-1\t",    "Parameter C\t ", "0.15\t",    "0.16\t",   "0.01\t", "6.7%\t"]
for Q2_y_Bin               in range(1, 18):
    for z_pT_Bin           in range(1, Get_Num_of_z_pT_Bins_w_Migrations(Q2_y_Bin_Num_In=Q2_y_Bin)[1] + 1):
        if(skip_condition_z_pT_bins(Q2_Y_BIN=Q2_y_Bin, Z_PT_BIN=z_pT_Bin, BINNING_METHOD="Y_bin")):
            continue
        Bin___Type__S = f"SMEARED Q2-y Bin {Q2_y_Bin} - z-PT Bin {z_pT_Bin}"
        Bin___Type_NS = f"Q2-y Bin {Q2_y_Bin} - z-PT Bin {z_pT_Bin}"
        for Par___Type in Parameters_to_Compare:
            Moment_Value__S = Full_Search_Parameter(Parameter_File_str=file_content, Bin_Info=Bin___Type__S, Histogram_Type=Histo_Type, Fit_Type=Fit___Type, Parameter=Par___Type)
            Moment_Value_NS = Full_Search_Parameter(Parameter_File_str=file_content, Bin_Info=Bin___Type_NS, Histogram_Type=Histo_Type, Fit_Type=Fit___Type, Parameter=Par___Type)
            Moment_Value__S = float(Moment_Value__S)
            Moment_Value_NS = float(Moment_Value_NS)
            Absolute_Dif = Moment_Value__S - Moment_Value_NS
            Percent__Dif = abs((abs(Absolute_Dif)/Moment_Value_NS)*100)
            Absolute_Dif = round(Absolute_Dif,       9)
            Percent__Dif = round(Percent__Dif,       9)
            if(Percent__Dif > 10):
                Percent__Dif = f"{color.RED}{Percent__Dif}"
            Moment_Value__S = round(Moment_Value__S, 9)
            Moment_Value_NS = round(Moment_Value_NS, 9)
            Comparison_Output[f"Par {Par___Type} - ({Q2_y_Bin}, {z_pT_Bin})"] = [f"Bin {Q2_y_Bin}-{z_pT_Bin}\t", f"Parameter {Par___Type}\t ", f"{Moment_Value__S}   \t", f"{Moment_Value_NS}   \t", f"{Absolute_Dif}   \t", f"{Percent__Dif}%  \t"]

print(f"\n{color.BOLD}Printing Table of Results for {color.UNDERLINE}({Histo_Type}) {Fit___Type}{color.END}")

full_table_print = ""
for ii in Comparison_Output:
    Line_Info = Comparison_Output[ii]
    if(ii in ["Title"]):
        print("==========================================================================================================================================")
    full_table_print = "".join([full_table_print, "\n", "" if(color.Error not in str(Line_Info[5])) else color.Error, f"{Line_Info[0]} || {Line_Info[1]} || {Line_Info[2]} || {Line_Info[3]}   || {Line_Info[4]} || {Line_Info[5]}\t||{color.END}"])
#     print("".join(["" if(color.Error not in str(Line_Info[5])) else color.Error, f"|| {Line_Info[0]} || {Line_Info[1]} || {Line_Info[2]} || {Line_Info[3]}   || {Line_Info[4]} || {Line_Info[5]}\t||{color.END}"]))
#     print("------------------------------------------------------------------------------------------------------------------------------------------")

full_table_print = full_table_print.replace("\t", "   ")
while("  " in full_table_print):
    full_table_print = full_table_print.replace("  ", " ")
    
full_table_print = full_table_print.replace("||", ";")
print(full_table_print)


Printing Table of Results for (1D) Bin-by-Bin Correction

Q2-y-z-PT Bin ; Fit Parameter ; Unsmeared ; Smeared ; Absolute Dif ; Percent Dif ;
Bin 1-1 ; Parameter B ; -0.059561591 ; -0.060466573 ; 0.000904982 ; 1.496665198% ;
Bin 1-1 ; Parameter C ; 0.014129271 ; 0.013698168 ; 0.000431102 ; 3.147153557% ;
Bin 1-2 ; Parameter B ; -0.055458603 ; -0.049584021 ; -0.005874582 ; 11.847731609% ;
Bin 1-2 ; Parameter C ; 0.051559526 ; 0.055078887 ; -0.003519362 ; 6.389675015% ;
Bin 1-3 ; Parameter B ; -0.113777981 ; -0.102847652 ; -0.010930329 ; 10.627689714% ;
Bin 1-3 ; Parameter C ; 0.043499065 ; 0.049000625 ; -0.00550156 ; 11.227529817% ;
Bin 1-4 ; Parameter B ; -0.159975657 ; -0.164159814 ; 0.004184158 ; 2.548831862% ;
Bin 1-4 ; Parameter C ; 0.029512493 ; 0.027863175 ; 0.001649318 ; 5.919346077% ;
Bin 1-5 ; Parameter B ; -0.222397197 ; -0.22384455 ; 0.001447352 ; 0.646588118% ;
Bin 1-5 ; Parameter C ; -0.001440129 ; -0.000125751 ; -0.001314379 ; 1045.225906061% ;
Bin 1-6 ; Parameter B ; -0.

.

.

## Compare Methods

In [169]:
Histo_Type = "1D"
Smear_Type = "SMEARED "
Smear_Type = ""
Parameters_to_Compare = ["B", "C"]

Fit___Type_to_Compare = ["Bin-by-Bin Correction", "Generated Plot"]
# Fit___Type_to_Compare = ["Bin-by-Bin Correction", "Bayesian Unfolding"]

Comparison_Output = {}
Comparison_Output["Title"] = ["Q2-y-z-PT Bin", "Fit Parameter",  f"{Fit___Type_to_Compare[0]}\t".replace("Correction", ""), f"{Fit___Type_to_Compare[1]}\t".replace("Correction", ""), "Absolute Dif\t", "Percent Dif\t\t"]
# Comparison_Output["Title2"] = ["Bin 18-35\t",  "Parameter B\t ", "0.15\t",    "0.16\t",   "0.01\t", "6.7%\t"]
# Comparison_Output["Title3"] = ["Bin 1-1\t",    "Parameter C\t ", "0.15\t",    "0.16\t",   "0.01\t", "6.7%\t"]
for Q2_y_Bin               in range(1, 18):
    for z_pT_Bin           in range(1, Get_Num_of_z_pT_Bins_w_Migrations(Q2_y_Bin_Num_In=Q2_y_Bin)[1] + 1):
        if(skip_condition_z_pT_bins(Q2_Y_BIN=Q2_y_Bin, Z_PT_BIN=z_pT_Bin, BINNING_METHOD="Y_bin")):
            continue
        Bin___Type = f"{Smear_Type}Q2-y Bin {Q2_y_Bin} - z-PT Bin {z_pT_Bin}"
        for Par___Type in Parameters_to_Compare:
            Moment_Value_F1 = Full_Search_Parameter(Parameter_File_str=file_content, Bin_Info=Bin___Type, Histogram_Type=Histo_Type, Fit_Type=Fit___Type_to_Compare[0], Parameter=Par___Type)
            Moment_Value_F2 = Full_Search_Parameter(Parameter_File_str=file_content, Bin_Info=Bin___Type, Histogram_Type=Histo_Type, Fit_Type=Fit___Type_to_Compare[1], Parameter=Par___Type)
            Moment_Value_F1 = float(Moment_Value_F1)
            Moment_Value_F2 = float(Moment_Value_F2)
            Absolute_Dif = Moment_Value_F1 - Moment_Value_F2
            Percent__Dif = abs((abs(Absolute_Dif)/Moment_Value_F2)*100)
            Absolute_Dif = round(Absolute_Dif,       9)
            Percent__Dif = round(Percent__Dif,       9)
            if(Percent__Dif > 25):
                Percent__Dif = f"{color.Error}{Percent__Dif}"
            Moment_Value_F1 = round(Moment_Value_F1, 9)
            Moment_Value_F2 = round(Moment_Value_F2, 9)
            Comparison_Output[f"Par {Par___Type} - ({Q2_y_Bin}, {z_pT_Bin})"] = [f"Bin {Q2_y_Bin}-{z_pT_Bin}\t", f"Parameter {Par___Type}\t ", f"{Moment_Value_F1}   \t", f"{Moment_Value_F2}   \t", f"{Absolute_Dif}   \t", f"{Percent__Dif}%  \t"]

print(f"\n{color.BOLD}Printing Table of Results between ({Histo_Type}) {color.UNDERLINE}{Fit___Type_to_Compare[0]}{color.END_B} and {color.UNDERLINE}{Fit___Type_to_Compare[1]}{color.END}")

full_table_print = ""
for ii in Comparison_Output:
    Line_Info = Comparison_Output[ii]
    if(ii in ["Title"]):
        print("==========================================================================================================================================")
    full_table_print = "".join([full_table_print, "\n", "" if(color.Error not in str(Line_Info[5])) else color.Error, f"{Line_Info[0]} || {Line_Info[1]} || {Line_Info[2]} || {Line_Info[3]}   || {Line_Info[4]} || {Line_Info[5]}\t||{color.END}"])
#     print("".join(["" if(color.Error not in str(Line_Info[5])) else color.Error, f"|| {Line_Info[0]} || {Line_Info[1]} || {Line_Info[2]} || {Line_Info[3]}   || {Line_Info[4]} || {Line_Info[5]}\t||{color.END}"]))
#     print("------------------------------------------------------------------------------------------------------------------------------------------")
    
full_table_print = full_table_print.replace("\t", "   ")
while("  " in full_table_print):
    full_table_print = full_table_print.replace("  ", " ")
    
full_table_print = full_table_print.replace("||", ";")
print(full_table_print)


Printing Table of Results between (1D) Bin-by-Bin Correction and Generated Plot

Q2-y-z-PT Bin ; Fit Parameter ; Bin-by-Bin ; Generated Plot ; Absolute Dif ; Percent Dif ;
Bin 1-1 ; Parameter B ; 0.002397199 ; 0.004371322 ; -0.001974123 ; 45.160776017% ;
Bin 1-1 ; Parameter C ; -0.003914275 ; -0.001535143 ; -0.002379132 ; 154.977855102% ;
Bin 1-2 ; Parameter B ; 0.00026262 ; -0.003819616 ; 0.004082236 ; 106.875568165% ;
Bin 1-2 ; Parameter C ; 0.007525246 ; 0.003664438 ; 0.003860808 ; 105.358794804% ;
Bin 1-3 ; Parameter B ; 0.00641398 ; -0.001748223 ; 0.008162203 ; 466.88577376% ;
Bin 1-3 ; Parameter C ; 0.004266343 ; -0.003862156 ; 0.008128499 ; 210.465310603% ;
Bin 1-4 ; Parameter B ; -0.004524674 ; 0.004646369 ; -0.009171044 ; 197.380854613% ;
Bin 1-4 ; Parameter C ; -0.008117244 ; -0.000768555 ; -0.00734869 ; 956.169728832% ;
Bin 1-5 ; Parameter B ; -0.005742178 ; -0.00480528 ; -0.000936898 ; 19.497266205% ;
Bin 1-5 ; Parameter C ; 0.002382784 ; 0.002483529 ; -0.000100744 ; 4.056

.

.

## Compare Dimensions/Sectors

In [8]:
# Histo_Type = "1D"
# Smear_Type = "SMEARED "
Smear_Type = ""
Fit___Type = "Bin-by-Bin Correction"
Parameters_to_Compare = ["B", "C"]

Histo_Type_to_Compare = ["1D", "1D (pipsec 1)", "1D (pipsec 2)", "1D (pipsec 3)", "1D (pipsec 4)", "1D (pipsec 5)", "1D (pipsec 6)"]
# Histo_Type_to_Compare = ["1D (pipsec 1)", "1D (pipsec 2)", "1D (pipsec 3)", "1D (pipsec 4)", "1D (pipsec 5)", "1D (pipsec 6)"]

Comparison_Output = {}
Comparison_Output_Title = ["Q2-y-z-PT Bin", "Fit Parameter"]
for compare in Histo_Type_to_Compare:
    Comparison_Output_Title.append(f"{compare}\t || {compare} Error")
Comparison_Output["Title"] = Comparison_Output_Title
del Comparison_Output_Title
# for Q2_y_Bin               in range(2, 18):
for Q2_y_Bin               in range(3, 4):
    for z_pT_Bin           in range(1, Get_Num_of_z_pT_Bins_w_Migrations(Q2_y_Bin_Num_In=Q2_y_Bin)[1] + 1):
        if(skip_condition_z_pT_bins(Q2_Y_BIN=Q2_y_Bin, Z_PT_BIN=z_pT_Bin, BINNING_METHOD="Y_bin")):
            continue
        Bin___Type = f"{Smear_Type}Q2-y Bin {Q2_y_Bin} - z-PT Bin {z_pT_Bin}"
        for Par___Type in Parameters_to_Compare:
            Line_to_Add = [f"Bin {Q2_y_Bin}-{z_pT_Bin}\t", f"Parameter {Par___Type}\t "]
            for Histo_Type in Histo_Type_to_Compare:
                Moment_Value_val, Moment_Value_err = Full_Search_Parameter(Parameter_File_str=file_content, Bin_Info=Bin___Type, Histogram_Type=Histo_Type, Fit_Type=Fit___Type, Parameter=Par___Type, return_err=True)
                if(Moment_Value_val not in ["ERROR"]):
                    Moment_Value_val = round(float(Moment_Value_val), 9)
                if(Moment_Value_err not in ["ERROR"]):
                    Moment_Value_err = round(float(Moment_Value_err), 9)
                Line_to_Add.append(f"{Moment_Value_val}   \t|| {Moment_Value_err}   \t")
            Comparison_Output[f"Par {Par___Type} - ({Q2_y_Bin}, {z_pT_Bin})"] = Line_to_Add

print(f"\n{color.BOLD}Printing Table of Results between ({Fit___Type}):")
for ii in Histo_Type_to_Compare:
    print(f"{color.UNDERLINE}{ii}{color.END_B}")

full_table_print = ""
for ii in Comparison_Output:
    Line_Info = Comparison_Output[ii]
#     print(f"Comparison_Output[{ii}] = {Line_Info}\n")
    if(ii in ["Title"]):
        print("==========================================================================================================================================")
    full_table_print = "".join([full_table_print, "\n", "||".join(Line_Info)])
#     print("".join(["" if(color.Error not in str(Line_Info[5])) else color.Error, f"|| {Line_Info[0]} || {Line_Info[1]} || {Line_Info[2]} || {Line_Info[3]}   || {Line_Info[4]} || {Line_Info[5]}\t||{color.END}"]))
#     print("------------------------------------------------------------------------------------------------------------------------------------------")
    
full_table_print = full_table_print.replace("\t", "   ")
while("  " in full_table_print):
    full_table_print = full_table_print.replace("  ", " ")
    
full_table_print = full_table_print.replace("||", ";")
print(full_table_print)


Printing Table of Results between (Bin-by-Bin Correction):
1D
1D (pipsec 1)
1D (pipsec 2)
1D (pipsec 3)
1D (pipsec 4)
1D (pipsec 5)
1D (pipsec 6)

Q2-y-z-PT Bin;Fit Parameter;1D ; 1D Error;1D (pipsec 1) ; 1D (pipsec 1) Error;1D (pipsec 2) ; 1D (pipsec 2) Error;1D (pipsec 3) ; 1D (pipsec 3) Error;1D (pipsec 4) ; 1D (pipsec 4) Error;1D (pipsec 5) ; 1D (pipsec 5) Error;1D (pipsec 6) ; 1D (pipsec 6) Error
Bin 3-1 ;Parameter B ;-0.07964092 ; 0.005797164 ;-0.108546432 ; 0.042640813 ;-0.08970482 ; 0.032680956 ;-0.088159 ; 0.030948547 ;-0.073628199 ; 0.029005331 ;-0.066007153 ; 0.047223191 ;-0.056655912 ; 0.04173308 
Bin 3-1 ;Parameter C ;0.019927132 ; 0.005604536 ;0.034930718 ; 0.025096736 ;0.042774777 ; 0.031917738 ;0.0453787 ; 0.034027434 ;-0.011111894 ; 0.00603265 ;0.042925027 ; 0.032193741 ;-0.001744472 ; 0.00130828 
Bin 3-2 ;Parameter B ;-0.104365833 ; 0.000288879 ;-0.024059778 ; 0.00353268 ;-0.014093565 ; 0.006070533 ;-0.120476761 ; 0.032109995 ;-0.178517183 ; 0.033547197 ;-0.22769962 

.

.

.

In [32]:
# import ROOT
# def create_random_histogram():
#     # Create a histogram with 100 bins ranging from -4 to 4
#     histogram = ROOT.TH1D("random_hist", "Random Gaussian", 10000, -4, 4)

#     # Fill the histogram with 1000 random numbers from a Gaussian distribution
#     for _ in range(1000000):
#         histogram.Fill(ROOT.gRandom.Gaus(0, 1))  # Mean = 0, Standard deviation = 1

#     # Set up a canvas to draw the histogram
#     canvas = ROOT.TCanvas("canvas", "Canvas for Histogram", 800, 600)
#     histogram.Draw()  # Draw the histogram on the canvas

#     # Update the canvas to show the histogram
#     canvas.Update()

#     return canvas, histogram  # Return both the canvas and the histogram for further use

# # Run the function to create and show the histogram
# canvas, histogram = create_random_histogram()
# canvas.Draw()

In [33]:
# import psutil
# import os

# # Function to get current process memory usage
# def get_memory_usage():
#     process = psutil.Process(os.getpid())  # Get current process
#     memory_info = process.memory_info()  # Get memory usage statistics
#     return memory_info.rss  # Return the Resident Set Size (RSS) which is the non-swapped physical memory the process has used


# import time
# # Example of using the function in a typical script workflow
# count = 0
# histo, canvas = {}, {}
# while(count < 5):  # Example of a loop, replace with your actual use case
#     # Do some work here
#     canvas[count], histo[count] = create_random_histogram()
#     canvas[count].SetName(f"canvas_{count}")
#     histo[count].SetName(f"histo_{count}")
#     canvas[count].Draw()
#     print(f"Current memory usage: {get_memory_usage() / (1024 * 1024):.2f} MB")  # Print memory usage in MB
# #     time.sleep(1)  # Sleep for a second, you can adjust or remove according to your needs
#     count += 1
# print("Done")

In [34]:
# command = "lt"
# command = f"{command}; echo ' '"
# for ii in range(1, 18):
# #     if(ii not in [7]):
#     command = f"{command}; echo 'Q2_y_Bin_{ii}:'"
#     command = f"{command}; rmdir Q2_y_Bin_{ii}/"
# #     command = f"{command}; ls -1 Q2_y_Bin_{ii}/ | wc -l"
# #     command = f"{command}; mv Q2_y_Bin_{ii}/*png ../../../../"
# #     command = f"{command}; ls -1 Q2_y_Bin_{ii}/ | wc -l"
# #     command = f"{command}; echo ' '"
# print(f"{command}")